# Summarize extreme datasets

This notebook is for creating summaries of the extreme variables derived from the CORDEX data. This will involve spreadsheets containing summarized data as well as some simple summary graphics.

### Eras

The eras of interest for summarizing these extremes will be:

* 2011-2040
* 2041-2070
* 2071-2100




In [3]:
import os
from pathlib import Path


# write to the final_products/auxiliary_content directory, as the outputs from here will be used for reporting
out_dir = Path(os.getenv("OUTPUT_DIR") or "/workspace/Shared/Tech_Projects/TBEC_CMIP5_Processing/final_products/")
extremes_fp = out_dir.joinpath("annual_extremes.nc")

Open connection to the extremes dataset:

In [4]:
import xarray as xr


ds = xr.open_dataset(extremes_fp)

## Excel spreadsheet

Create an excel spreadsheet of tidy tables of summarized extreme variables, where each worksheet is one of the study locations. Each tidy table will have the following headers:

* `model`
* `scenario`
* `era` (time period)
* `variable`

Create some iterables for extracting the various summaries:

In [33]:
eras = [
    "1981-2010",
    "2011-2040",
    "2041-2070",
    "2071-2100",
]
# only need to iterate over these two scenarios, since 
#  "historical" era actually contains 5 years from 
#  either of the future scenarios
scenarios = ["rcp45", "rcp85"]
# dict of WGS84 coords for each of the locations
locations = {
    "Kaktovik": (70.1, -143.6),
    "Stevens Village": (66.1, -149.1),
    "Igiugik Village": (59.3, -155.9),
    "Levelock": (59.1, -156.9),
    "Nelson Lagoon": (56.0, -161.2),
    "Eyak": (60.5, -145.6),
    "Ketchikan": (55.6, -136.6),
    "Unalaska": (53.9, -166.5),
}

In [ ]:
for model in ds.model.values:
    for scenario in scenarios:
        